In [2]:
import json
import numpy as np
import networkx as nx
from scipy.optimize import linprog


In [3]:
thresholds = {}

In [4]:
with open("data/thresholds.txt") as f:
    for line in f:
        name, th = line.split()
        thresholds[name] = int(th)

In [5]:
def check_cover(g, cover):
    for n in g.edges():
        if int(n[0]) not in cover and int(n[1]) not in cover:
            print(f"Edge {n} not covered")
            return False
    return True

In [6]:
def find_cover_max_matching(g):
    """
        2-approxmation via maximal matching
    """
    cover = set()

    for u, v in g.edges():
        if u not in cover and v not in cover:
            cover.add(u)
            cover.add(v)

    return list([int(c) for c in cover])

In [7]:
def find_cover_lp(g):
    """
        2-approximation using Linear Programming
    """
    # formulate Vertex Cover as Set Cover
    universe = list(g.edges())
    sets = [[e for e in universe if e[0] == n] for n in g.nodes()]

    # cost
    c = np.ones(len(sets), dtype=np.float64)

    # inequality constraints
    b = -np.ones(len(universe), dtype=np.float64)

    A = np.zeros((len(universe), len(sets)))

    for ie, e in enumerate(universe):      
        for i, s in enumerate(sets):
            if e in s:
                A[ie, i] = -1

    res = linprog(c, A_ub=A, b_ub=b)

    if not res.success:
        raise RuntimeError(res.message)

    z = np.array(res.x)
    return list(np.where(z >= 1./2.)[0] + 1)

In [8]:
def find_cover_greedy(g):
    """
        Greedy implementation based on networkx (2-approx)
    """
    edges = list(g.edges())   
    cost = {n: 1 for n in g.nodes()}

    for u, v in edges:
        min_cost = min(cost[u], cost[v])
        cost[u] -= min_cost
        cost[v] -= min_cost
        
    cover = {u for u, c in cost.items() if c == 0}
    return sorted([int(c) for c in cover])

In [9]:
def find_cover_max_degree(g):
    """
        Max-degree heuristics
    """

    dgs = g.degree()
    cover = set()

    for u, v in g.edges():
        if u not in cover and v not in cover:
            if dgs(u) > dgs(v):
                cover.add(u)
            else:
                cover.add(v)

    return sorted([int(c) for c in cover])

In [10]:
def optimize(g, cover):
    # optimization
    while True:
        ll = list(cover)
        cover = set()
        # optimization
        for n in ll:
            ne = g.edges(str(n+1))
            for e in ne:
                o = e[1]
                if o not in cover:
                    cover.add(n)
                    break
        if len(ll) == len(cover):
            break
    return cover

In [11]:
result = {}
for name, th in thresholds.items():

    with open(f"data/{name}") as f:
        f.readline()
        g = nx.read_edgelist(f, comments="c")

    # cover_lp = find_cover_lp(g)
    # cover_greedy = find_cover_greedy(g)
    # cover_mm = find_cover_max_matching(g)
    # cover = optimize(g, cover)

    cover = find_cover_max_degree(g)

    if len(cover) <= th:
        print(name, "pass")
        result[name] = cover
    else:
        print(f"{name} fail : size={len(g.nodes())} cover={len(cover)} th={th}")

    # if not check_cover(g, cover):
    #   break

vc-exact_001.gr fail : size=6160 cover=2614 th=2606
vc-exact_003.gr pass
vc-exact_005.gr fail : size=200 cover=150 th=136
vc-exact_007.gr fail : size=8794 cover=4908 th=4812
vc-exact_009.gr pass
vc-exact_011.gr fail : size=9875 cover=5085 th=5008
vc-exact_013.gr pass
vc-exact_015.gr pass
vc-exact_017.gr fail : size=23541 cover=13306 th=12769
vc-exact_019.gr fail : size=200 cover=144 th=136
vc-exact_021.gr pass
vc-exact_023.gr pass
vc-exact_025.gr pass
vc-exact_027.gr pass
vc-exact_029.gr fail : size=13431 cover=7204 th=7161
vc-exact_031.gr fail : size=200 cover=152 th=145
vc-exact_033.gr pass
vc-exact_035.gr fail : size=200 cover=154 th=140
vc-exact_037.gr fail : size=198 cover=145 th=138
vc-exact_039.gr pass
vc-exact_041.gr fail : size=200 cover=154 th=147
vc-exact_043.gr fail : size=200 cover=161 th=146
vc-exact_045.gr fail : size=200 cover=155 th=148
vc-exact_047.gr fail : size=200 cover=153 th=146
vc-exact_049.gr fail : size=200 cover=150 th=145
vc-exact_051.gr fail : size=200 cove

In [46]:
with open("result.json", "w") as f:
    json.dump(result, f, indent=2)